In [ ]:
!pip install -q tensorflow-recommenders
!pip install -q --upgrade tensorflow-datasets
!pip install -q scann

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.2/96.2 kB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.6/9.6 MB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 479.7/479.7 MB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 58.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 56.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 440.8/440.8 kB 33.9 MB/s eta 0:00:00


In [ ]:
import os
import pprint
import tempfile

from typing import Dict, Text

import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds
import tensorflow_recommenders as tfrs
from sklearn import preprocessing
from sklearn.preprocessing import MinMaxScaler
from sklearn.neighbors import NearestNeighbors
import html
# pd.set_option('display.max_columns', None)
# pd.set_option('display.max_rows', None)
# pd.set_option('display.expand_frame_repr', False)

In [ ]:
from google.colab import files

# Mengunggah file dari komputer lokal ke Colab
uploaded = files.upload()

# Menampilkan informasi tentang file yang diunggah
for filename in uploaded.keys():
    print(f'File {filename} berhasil diunggah dengan ukuran {len(uploaded[filename])} bytes')

Saving cleaned_recipes.xlsx to cleaned_recipes.xlsx
File cleaned_recipes.xlsx berhasil diunggah dengan ukuran 7379416 bytes


In [ ]:
import pandas as pd

# Replace 'your_file.xlsx' with the actual file path of your Excel file.
df = pd.read_excel('cleaned_recipes.xlsx')
df

,id_makanan,nama,kalori,protein,bahan,cooking_step
0,41,Carina's Tofu-Vegetable Kebabs,536.1,29.3,"extra firm tofu, eggplant, zucchini, mushrooms...","Drain the tofu, carefully squeezing out excess..."
1,64,Almond Pound Cake,408.5,6.9,"butter, almond paste, sugar, eggs, flour, corn...","Preheat oven to 350 degrees Fahrenheit., Cream..."
2,94,Blueberry Buttertarts,337.9,4.2,"egg, corn syrup, lemon juice, vanilla, brown s...","Preheat oven to 375F., Leave shells in foil cu..."
3,131,Buttermilk-Oat Bread,68.7,2.1,"buttermilk, honey, butter, margarine, bread fl...","For toasted rolled oats, place oats in a shall..."
4,168,Anzac Biscuits,199.4,2.2,"rolled oats, plain flour, sugar, golden syrup,...","Mix oats, flour, sugar and coconut together., ..."
...,...,...,...,...,...,...
22712,539056,Belgian Tomato Meatball Soup (Tomatensoep Met ...,192.8,8.5,"chopped tomato, fresh tomatoes, fresh carrot, ...",Add the butter to a large and high pan. Place ...
22713,539304,Pork Tenderloin With Bacon-Onion Jam,558.4,33.2,"olive oil, red onion, bacon, fresh thyme, brow...",For the bacon-onion jam: Heat the oil in a cas...
22714,539476,Thai Basil Chicken (Gai Pad Krapow),300.5,22.6,"shallots, garlic cloves, ground chicken, sugar...","In a wok over high heat, add the oil, chilies,..."
22715,540238,Keto King's Best Bread,50.4,2.0,"flax seed meal, vital wheat gluten, xanthan gu...","Combine yeast, honey and warm water., Allow to..."


In [ ]:
columns = df.columns.values
columns

array(['id_makanan', 'nama', 'kalori', 'protein', 'bahan', 'cooking_step'],
      dtype=object)

In [ ]:
df.isna().sum()

id_makanan      0
nama            0
kalori          0
protein         0
bahan           0
cooking_step    0
dtype: int64

In [ ]:
# simple preprocess df['nama']
df['nama'] = df['nama'].apply(html.unescape)

# Features Kalorize and KNN Prediction

In [ ]:
df_copy = df.copy()

df_features = df.iloc[:, 2:4]
min_max_scaler = MinMaxScaler()
df_features = min_max_scaler.fit_transform(df_features)
all_calories = list(df['kalori'].values)

In [ ]:
model = NearestNeighbors(n_neighbors=10, algorithm='ball_tree').fit(df_features)
# KNN return 2 array yaitu distances dan indices
indices = model.kneighbors(df_features, return_distance=False)

In [ ]:
def closest(all_calories, input):
    return all_calories[min(range(len(all_calories)), key=lambda i: abs(all_calories[i]-input))]

In [ ]:
def calculate_bmr(gender, weight, height, age):
    if gender == 'male':
        bmr = 10 * weight + 6.25 * height - 5 * age + 5
    elif gender == 'female':
        bmr = 10 * weight + 6.25 * height - 5 * age - 161
    else:
        raise ValueError("Invalid gender. Please choose 'male' or 'female'.")
    return bmr

In [ ]:
def calculate_total_calories(bmr, activity_level, target):
    activity_levels = {
        'easy': 1.375,
        'medium': 1.55,
        'hard': 1.725,
        'extreme': 1.9
    }
    targets = {
        'reduce_weight': 0.8,
        'increase_muscle': 1.2,
        'be_healthy': 1
    }
    maintain_calories = bmr * activity_levels[activity_level]
    total_calories = maintain_calories * targets[target]
    return total_calories

In [ ]:
def recommendation(gender, weight, height, age, activity_level, target):
    bmr = calculate_bmr(gender, weight, height, age)
    total_calories = calculate_total_calories(bmr, activity_level, target)

    breakfast_calories = total_calories * 0.35
    lunch_calories = total_calories * 0.40
    dinner_calories = total_calories * 0.25

    breakfast_closest_food = closest(all_calories, breakfast_calories)
    lunch_calories_food = closest(all_calories, lunch_calories)
    dinner_calories_food = closest(all_calories, lunch_calories)

    breakfast_food_code = all_calories.index(breakfast_closest_food)
    lunch_food_code = all_calories.index(lunch_calories_food)
    dinner_food_code = all_calories.index(dinner_calories_food)

    breakfast_similarity = df.iloc[indices[breakfast_food_code]]
    lunch_similarity = df.iloc[indices[lunch_food_code]]
    dinner_similarity = df.iloc[indices[dinner_food_code]]

    return breakfast_similarity, lunch_similarity, dinner_similarity

In [ ]:
gender = 'male'
weight = 52
height = 169
age = 20
activity_level = 'medium'
target = 'reduce_weight'

breakfast, lunch, dinner = recommendation(
    gender, weight, height, age, activity_level, target)

In [ ]:
print(breakfast)
print(lunch)
print(dinner)

       id_makanan                                               nama  kalori  protein                                              bahan                                       cooking_step
816         22562                                      Tiger Waffles   642.8     13.2                                   water, margarine  Change the angle on the George Foreman meat co...
8117       267860                          Kifli  (Crescent Cookies)   647.2     12.9  flour, butter, margarine, dry yeast, milk, sug...  DOUGH:, Blend flour and butter thoroughly usin...
21250      478799  Roasted Eggplant With Harissa Potatoes and Tom...   656.8     12.6  eggplant, potatoes, tomatoes, water, tomato pa...  Preheat oven to 420 degrees F., Peel eggplant ...
15039      374320                        Gram Edna's Fluffy Pancakes   635.1     14.2       flour, baking powder, sugar, salt, egg, milk  Combine Oil, Egg, and Milk., Add remaining ing...
14657      368504                          Lemon Syrup Panca

# Data Using Tensorflow

In [ ]:
food_data = tf.data.Dataset.from_tensor_slices({
    "id_makanan": df["id_makanan"].values,
    "nama": df["nama"].values,
})

food_ingridient = tf.data.Dataset.from_tensor_slices({
    "nama": df["nama"].values,
    "kalori": df["kalori"].values,
    "protein": df["protein"].values,
    'bahan': df['bahan'].values,
    'cooking_step': df['cooking_step'].values
})

food_name = tf.data.Dataset.from_tensor_slices({
    "nama": df["nama"].values,
})

tf.random.set_seed(42)
shuffled = food_data.shuffle(100_000, seed=42, reshuffle_each_iteration=False)

train = food_data.take(80_000)
test = food_data.skip(80_000).take(20_000)

In [ ]:
food_titles = food_name.batch(1_000)
food_titles_arrays = food_titles.map(lambda x: x['nama'])

food_ids = food_data.batch(1_000_000).map(lambda x: x["id_makanan"])
food_ids_np = np.array([x.numpy().tolist() for x in food_ids])
unique_food_titles = np.unique(np.concatenate(list(food_titles_arrays)))
unique_food_ids = np.unique(np.concatenate(list(food_ids)))
unique_food_ids = [str(id) for id in unique_food_ids]

unique_food_titles[:10]

array([b'" Berry French Toast" Oatmeal',
       b'" Better Than Cinnabon" Cinnamon Rolls',
       b'" Chinese"  Sweet and Sour Balls', b'" Hot Tamale" Burgers',
       b'" I Thought I Had Nothing to Eat" Rice',
       b'" Jamba Juice at Home" Strawberries Wild  Smoothie',
       b'" Lemon Coconut Custard" Pie', b'" Lite" Cretons',
       b'" Loaded " Deviled Eggs', b'" Ma\'s" Oatmeal Cake and Icing'],
      dtype=object)

# Model

In [ ]:
embedding_dimension = 32

In [ ]:
food_model = tf.keras.Sequential([
  tf.keras.layers.StringLookup(
      vocabulary=unique_food_ids, mask_token=None),
  # We add an additional embedding to account for unknown tokens.
  tf.keras.layers.Embedding(len(unique_food_ids) + 1, embedding_dimension)
])

In [ ]:
food_name_model = tf.keras.Sequential([
  tf.keras.layers.StringLookup(
      vocabulary=unique_food_titles, mask_token=None),
  tf.keras.layers.Embedding(len(unique_food_titles) + 1, embedding_dimension)
])

In [ ]:
metrics = tfrs.metrics.FactorizedTopK(
  candidates=food_titles_arrays.batch(128).map(food_name_model)
)

In [ ]:
task = tfrs.tasks.Retrieval(
  metrics=metrics
)

In [ ]:
class FoodslensModel(tfrs.Model):

  def __init__(self, food_model, food_name_model):
    super().__init__()
    self.food_name_model: tf.keras.Model = food_name_model
    self.food_model: tf.keras.Model = food_model
    self.task: tf.keras.layers.Layer = task

  def compute_loss(self, features: Dict[Text, tf.Tensor], training=False) -> tf.Tensor:
    # We pick out the user features and pass them into the user model.
    user_embeddings = self.food_model(features["id_makanan"])
    # And pick out the movie features and pass them into the movie model,
    # getting embeddings back.
    positive_movie_embeddings = self.food_name_model(features["nama"])

    # The task computes the loss and the metrics.
    return self.task(user_embeddings, positive_movie_embeddings)

In [ ]:
class NoBaseClassFoodlensModel(tf.keras.Model):

  def __init__(self, food_model, food_name_model):
    super().__init__()
    self.food_name_model: tf.keras.Model = food_name_model
    self.food_model: tf.keras.Model = food_model
    self.task: tf.keras.layers.Layer = task

  def train_step(self, features: Dict[Text, tf.Tensor]) -> tf.Tensor:

    # Set up a gradient tape to record gradients.
    with tf.GradientTape() as tape:

      # Loss computation.
      user_embeddings = self.food_model(features["id_makanan"])
      positive_movie_embeddings = self.food_name_model(features["nama"])
      loss = self.task(user_embeddings, positive_movie_embeddings)

      # Handle regularization losses as well.
      regularization_loss = sum(self.losses)

      total_loss = loss + regularization_loss

    gradients = tape.gradient(total_loss, self.trainable_variables)
    self.optimizer.apply_gradients(zip(gradients, self.trainable_variables))

    metrics = {metric.name: metric.result() for metric in self.metrics}
    metrics["loss"] = loss
    metrics["regularization_loss"] = regularization_loss
    metrics["total_loss"] = total_loss

    return metrics

  def test_step(self, features: Dict[Text, tf.Tensor]) -> tf.Tensor:

    # Loss computation.
    user_embeddings = self.food_model(features["id_makanan"])
    positive_movie_embeddings = self.food_name_model(features["nama"])
    loss = self.task(user_embeddings, positive_movie_embeddings)

    # Handle regularization losses as well.
    regularization_loss = sum(self.losses)

    total_loss = loss + regularization_loss

    metrics = {metric.name: metric.result() for metric in self.metrics}
    metrics["loss"] = loss
    metrics["regularization_loss"] = regularization_loss
    metrics["total_loss"] = total_loss

    return metrics

In [ ]:
model = FoodslensModel(food_model, food_name_model)
model.compile(optimizer=tf.keras.optimizers.Adagrad(learning_rate=0.1))

In [ ]:
cached_train = train.shuffle(100_000).batch(8192).cache()
cached_test = test.batch(4096).cache()